# Viet Nam traffic sign  detection with dataset Yolov8 x10

## SETUP VÀ CÀI ĐẶT THƯ VIỆN

In [ ]:

!pip install -q ultralytics albumentations opencv-python

import os
import cv2
import albumentations as A
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO

import yaml
import shutil
from pathlib import Path

import torch
import matplotlib.pyplot as plt
from IPython.display import Image, display
import pandas as pd



## ĐƯỜNG DẪN

In [ ]:
DATASET_DIR = "/kaggle/input/vietnamses-traffic-sign-detection-augmentaion/dataset"  # 🟡 Thay đổi tên thư mục nếu cần
WORK_DIR = "/kaggle/working/dataset"

# Sao chép dữ liệu sang working directory (để có quyền ghi)
!cp -r {DATASET_DIR} {WORK_DIR}



In [ ]:
"""
    Nếu hong có data.yaml hoặc bị lỗi thì dùng
"""

data_yaml = f"""
path: {WORK_DIR}
train: images/train
val: images/val
test: images/test

nc: 29
names:
  0: one way prohibition
  1: no parking
  2: no stopping and parking
  3: no turn left
  4: no turn right
  5: no u turn
  6: no u and left turn
  7: no u and right turn
  8: no motorbike entry/turning
  9: no car entry/turning
  10: no truck entry/turning
  11: other prohibition
  12: indication
  13: direction
  14: speed limit
  15: weight limit
  16: height limit
  17: pedestrian crossing
  18: intersection danger
  19: road danger
  20: pedestrian danger
  21: construction danger
  22: slow warning
  23: other warning
  24: vehicle permission lane
  25: vehicle and speed permission lane
  26: overpass route
  27: no more prohibition
  28: other
"""

with open(os.path.join(WORK_DIR, "data.yaml"), "w") as f:
    f.write(data_yaml)


In [ ]:
"""
    Liệt kê 10 file trong 1 folder 
"""

def list_10_file(input_dir):
    for dirname, _, filenames in os.walk(input_dir):
        if filenames: # nếu thư mục này có file
            print(f'Folder: {dirname}')
            for file in filenames[:10]:
                print(f'File: {file}')
        print() # xuống dòng

# input_dir = '/kaggle/input'
# list_10_file(input_dir)

## Hyperparameter và Train model

In [ ]:

"""
    Train lần đầu 
"""

# Cấu hình Accelerator
if torch.cuda.is_available():
    device = '0,1' if torch.cuda.device_count() > 1 else '0'
else:
    device = 'cpu'

TRAINING_CONFIG = {
    'epochs': 150,
    'imgsz': 640,
    'batch': 32,  # Giảm xuống nếu bị out of memory
    'lr0': 0.01,
    'lrf': 0.005,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'patience': 10,  # Early stopping patience
    'save_period': 10,  # Save checkpoint every N epochs
    'device': device
}

print("✅ Thiết bị đang sử dụng:", TRAINING_CONFIG['device'])
print("🧠 Số GPU khả dụng:", torch.cuda.device_count())

model = YOLO("yolov8m.pt")  # hoặc yolov8s.pt, yolov8m.pt tùy chọn

results = model.train(
    data=os.path.join(WORK_DIR, "data.yaml"),

    epochs=TRAINING_CONFIG['epochs'],
    imgsz=TRAINING_CONFIG['imgsz'],
    batch=TRAINING_CONFIG['batch'],
    lr0=TRAINING_CONFIG['lr0'],
    lrf=TRAINING_CONFIG['lrf'],
    momentum=TRAINING_CONFIG['momentum'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    warmup_epochs=TRAINING_CONFIG['warmup_epochs'],
    warmup_momentum=TRAINING_CONFIG['warmup_momentum'],
    warmup_bias_lr=TRAINING_CONFIG['warmup_bias_lr'],
    patience=TRAINING_CONFIG['patience'],
    save_period=TRAINING_CONFIG['save_period'],
    device=TRAINING_CONFIG['device'],

    name='yolov8_experiment',
    exist_ok=True,
    pretrained=True,
    optimizer='SGD',
    verbose=True,
    seed=42,
    deterministic=True,
    single_cls=False,
    rect=False,
    cos_lr=True,
    close_mosaic=10,
    resume=False,
    amp=True,
    fraction=1.0,
    profile=False,
    overlap_mask=True,
    mask_ratio=4,
    dropout=0.0,
    val=True,
    plots=True,
    save=True,
    save_json=False
)


In [ ]:
"""
    Cell chạy lại epoch thứ n bị dừng trong tổng epoch

    Các chỉ số không nên thay đổi so với session trước: lr0, lrf, momentum, weight_decay, 
    warmup_epochs, warmup_momentum, warmup_bias_lr, name, optimizer, pretrained.
    Các chỉ số còn lại có thể chỉnh thêm
"""

# Cấu hình Accelerator
if torch.cuda.is_available():
    device = '0,1' if torch.cuda.device_count() > 1 else '0'
else:
    device = 'cpu'

TRAINING_CONFIG = {
    'epochs': 150,
    'imgsz': 640,
    'batch': 32,  # Giảm xuống nếu bị out of memory
    'lr0': 0.01,
    'lrf': 0.005,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'patience': 10,  # Early stopping patience
    'save_period': 10,  # Save checkpoint every N epochs
    'device': device
}

print("✅ Thiết bị đang sử dụng:", TRAINING_CONFIG['device'])
print("🧠 Số GPU khả dụng:", torch.cuda.device_count())

# Load model đã train trước đó
model = YOLO("/kaggle/input/model-110th/pytorch/default/1/epoch110.pt")  # Hoặc đường dẫn khác nếu bạn lưu nơi khác

# Tiếp tục train
results = model.train(
    data=os.path.join(WORK_DIR, "data.yaml"),

    epochs=TRAINING_CONFIG['epochs'],
    imgsz=TRAINING_CONFIG['imgsz'],
    batch=TRAINING_CONFIG['batch'],
    lr0=TRAINING_CONFIG['lr0'],
    lrf=TRAINING_CONFIG['lrf'],
    momentum=TRAINING_CONFIG['momentum'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    warmup_epochs=TRAINING_CONFIG['warmup_epochs'],
    warmup_momentum=TRAINING_CONFIG['warmup_momentum'],
    warmup_bias_lr=TRAINING_CONFIG['warmup_bias_lr'],
    patience=TRAINING_CONFIG['patience'],
    save_period=TRAINING_CONFIG['save_period'],
    device=TRAINING_CONFIG['device'],

    name='yolov8_experiment',
    resume=True,
    exist_ok=True,
    pretrained=True,
    optimizer='SGD',
    verbose=True,
    seed=42,
    deterministic=True,
    single_cls=False,
    rect=False,
    cos_lr=True,
    close_mosaic=10,
    amp=True,
    fraction=1.0,
    profile=False,
    overlap_mask=True,
    mask_ratio=4,
    dropout=0.0,
    val=True,
    plots=True,
    save=True,
    save_json=False
)


In [ ]:
"""
    Cell để train thêm khi model đã là last.pt 
"""

model = YOLO("/kaggle/input/yolo_epochs100th/pytorch/default/1/last.pt")  

results = model.train(
    data=os.path.join(WORK_DIR, "data.yaml"),
    epochs=50,  # Train thêm 50 epochs 
    device=device,
    name='yolov8_experiment_v2',  # Đặt tên mới để tránh ghi đè
    exist_ok=True,
    val=True,
    resume=False  # Không resume nữa
)


## Visualize

In [ ]:
# Đường dẫn thư mục lưu kết quả
result_dir = '/kaggle/working/runs/detect/yolov8_experiment'
# result_dir = '/kaggle/working/runs/detect/yolov8_experiment_v2'


# File CSV chứa log từng epoch
csv_log_path = '/kaggle/working/runs/detect/yolov8_experiment/results.csv'
# csv_log_path = '/kaggle/working/runs/detect/yolov8_experiment_v2/results.csv'

In [ ]:
# Danh sách các biểu đồ
plots = [
    'results.png',              # Tổng hợp các chỉ số: box loss, cls loss, precision, recall, mAP
    'confusion_matrix.png',
    'F1_curve.png',
    'P_curve.png',
    'R_curve.png',
    'PR_curve.png'
]

# Hiển thị từng biểu đồ
for plot in plots:
    plot_path = os.path.join(result_dir, plot)
    if os.path.exists(plot_path):
        print(f"Biểu đồ: {plot}")
        display(Image(filename=plot_path))
    else:
        print(f"Không tìm thấy {plot}")


In [ ]:
# Đọc file kết quả
df = pd.read_csv(csv_log_path)  # Đường dẫn tùy vào nơi bạn train

# Hiển thị các cột có sẵn
print("Các cột có trong CSV:", df.columns.tolist())

# --- Vẽ Loss ---
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss')
plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss')
plt.plot(df['epoch'], df['train/cls_loss'], label='Train Cls Loss')
plt.plot(df['epoch'], df['val/cls_loss'], label='Val Cls Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss theo Epoch')
plt.legend()

# --- Vẽ Precision, Recall ---
plt.subplot(1, 3, 2)
plt.plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
plt.plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
plt.xlabel('Epoch')
plt.ylabel('Tỷ lệ')
plt.title('Precision & Recall theo Epoch')
plt.legend()

# --- Vẽ mAP ---
plt.subplot(1, 3, 3)
plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP50')
plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP50-95')
plt.xlabel('Epoch')
plt.ylabel('Tỷ lệ')
plt.title('mAP theo Epoch')
plt.legend()

plt.tight_layout()
plt.show()

# --- In model tốt nhất ---
best_epoch = df['metrics/mAP50-95(B)'].idxmax()
best_map = df.loc[best_epoch]
print(f"\n📌 Model tốt nhất nằm ở epoch {int(best_map['epoch'])}")
print(f"mAP50-95 = {best_map['metrics/mAP50-95(B)']:.4f}")
print(f"mAP50    = {best_map['metrics/mAP50(B)']:.4f}")
print(f"Precision = {best_map['metrics/precision(B)']:.4f}")
print(f"Recall    = {best_map['metrics/recall(B)']:.4f}")


In [ ]:
folder = '/kaggle/working/runs/detect/yolov8_experiment'

# Lấy tất cả file .png và .jpg
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.png', '.jpg'))]

# Hiển thị từng hình
for f in files:
    display(Image(filename=f))
